In [1]:
#!/usr/bin/env python3
 #-*- coding: utf-8 -*-
import glob
import os
import pickle
import sys
from pathlib import Path
import random
from tqdm import tqdm

import torchvision.models as models
from torch.nn import Module, Dropout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import torchvision

#from pytorchtools import EarlyStopping

from sklearn.metrics import mean_squared_error
import pandas as pd

from torchvision import datasets, models, transforms
import time
import logging

In [2]:
class VGG_19_Regression(Module):
    
    def __init__(self,):
        
        model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19_bn', pretrained=True)
        
        super(VGG_19_Regression,self).__init__()
        self.model= model
        self.Regression = nn.Linear(1000,1)
        self.dropout = Dropout(dropout=0.5)
        
           
         
    def forward(self, out):
        
        x= self.model(out)
        
        x = x.view(x.size(0), -1) # flatten
        
        x = self.dropout(x)
        
        x =self.Regression(x)
        
        return x

In [14]:



def training(epoch):
     
        
    train_transforms = transforms.Compose([
                                    transforms.Resize(224),
                                    transforms.RandomVerticalFlip(),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ColorJitter(brightness=0, contrast=0, saturation=1, hue=.5),
                                    transforms.ToTensor()])


    val_transforms= transforms.Compose([
                                    transforms.Resize(224),
                                    transforms.ToTensor()])



    # cross validaion with 5 folders on the train 
    train_dataset = datasets.ImageFolder('/mnt/scratchc/fmlab/eudari01/BREAST_METABRIC/tiles_CIN', train_transforms) 

    # cross validaion with 5 folders on the val 
    val_dataset = datasets.ImageFolder('/mnt/scratchc/fmlab/eudari01/BREAST_METABRIC/tiles_CIN', val_transforms) 


    batch_size = 48
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True)
    valid_loader= torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=16, shuffle=True)

    Model= VGG_19_Regression()
    MSE = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(Model.parameters(), lr=0.00001,momentum=0.9)  
    
    for i in range(5) : 
        
        
        for layer in Model.children():
                   if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
        Model.train()              
        epoch_loss = 0.0
        running_loss_train=0.0
   
        for data, label in tqdm(train_loader):
        
                  label= label.to(torch.float32)
                  label = label.unsqueeze(1)

         
        #Transfer Data to GPU if available
            #if torch.cuda.is_available():
            
            
                #data = data.cuda()
                #label = label.cuda()
            
                  output = Model(data)
            #you can change the loss and use MSE or Cross Entropy
                  loss = MSE(output,label)
           
            # Clear the gradients
                  optimizer.zero_grad()
            
                  loss.backward()
                  optimizer.step()
                
            #Calculate accuracy 
            #prediction_train=output.argmax(dim=1)
           # acc = (prediction_train== label1).float().mean()
        
                  running_loss_train =+ loss.item()* data.size(0) 
            #Epoch
            #epoch_accuracy += acc.item() / len(train_loader)
                  epoch_loss += running_loss_train / len(train_loader)
                  r2_score_train=r2_score(label, prediction_train)
            
            
        Model.eval()
        epoch_val_loss = 0.0
        running_loss_val= 0.0
        min_val_loss=0.0
        epochs_no_improve =0
        n_epochs_stop = 20
        
        for layer in Model.children():
                   if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
        
        
        for data, label in valid_loader:
            
                        output = Model(data)
                
                        loss = MSE(output, label)
                
                # Clear the gradients
                #optimizer.zero_grad()
                
                #val_loss.backward()
                
                #optimizer.step()
                #Calculate accuracy 
                #prediction_val= val_output.argmax(dim=1)
                
                #mse_val=mean_squared_error(label,prediction_val)
                
                #val_acc = (prediction_val == label).float().mean()
                
                        running_loss_val =+ loss.item()* data.size(0) 
               
                #Epoch
                #epoch_val_accuracy += val_acc.item() / len(valid_loader)
                        epoch_val_loss += running_loss_val / len(valid_loader)
                        r2_score_val= r2_score(label, prediction_val)
        
        #to resset the train and validation for each split
        for layer in Model.children():
                   if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
        
        
        
        if  running_loss_val < min_val_loss:
                      # Save the model
                torch.save(Model)
                #epochs_no_improve = 0
                #n_epochs_stop = 20
                min_val_loss = running_loss_val
  
        else:
                epochs_no_improve += 1
        iter += 1
                
        if epochs_no_improve == n_epochs_stop:
             print('Early stopping!')
            

             break
               
        else:
            continue
                
        
            
        val_loss.append(epoch_val_loss)
        train_loss.append(epoch_loss)
    #train_accuracy.append(epoch_accuracy)
        #print(
            #f"Epoch : {epoch+1} \n")
        print('Split: {}'.format(i,epoch_loss,epoch_val_loss,r2_score_train, r2_score_val))                
        print(
            f"Epoch : {epoch+1} - train_loss : {epoch_loss:.4f} - val_loss : {epoch_val_loss:.4f}- r2_score_train : {r2_score_train:.4f}- r2_score_val : {r2_score_val:.4f} \n")

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/scratchc/fmlab/eudari01/BREAST_METABRIC/tiles_CIN'

In [22]:
def runmymodel2(job):
    #CREATE OUTPUT DIR
    
    
    
    
    train_loss = []
    val_loss = []
    r2_score_train=[]
    r2_score_val= []

    labeltouse = 'CIN'
    #input_path = '/data/processed_inputs/'
    #input_path2 = '/data/cluster_data_v3/'
    HPs = pd.read_csv('/mnt/scratchc/fmlab/eudari01/BREAST_METABRIC/80_SAGE_CIN_HP.txt ')
    myhp = HPs.iloc[int(job)-1]
    lr = myhp.lr
    momentum=myhp.momentum
    dropout = myhp.dropout
    batch_size = myhp.batch_size 
    
    
    output_path = '/mnt/scratchc/fmlab/eudari01/BREAST_METABRIC/CIN_results'+labeltouse  + '_lr_'+str(lr)+ '_momentum_'+str(momentum)+'_dropout_'+str(dropout)+'_batch_size_'+str(batch_size)+'/'
    try:
        os.mkdir(output_path)
    except:
        print('Directory already exists!')

    print('creating dirs ...')
    print('starting CV training')
    
    
    epochs=30
    for epoch in range(1,epochs+1):
         training(epoch)
    
    
    results_df1 = pd.DataFrame()
    
    nametosave_epoch = 'CIN_'+labeltouse  + '_lr_'+str(lr)+ '_momentum_'+str(momentum)+'_dropout_'+str(dropout)+'_batch_size_'+str(batch_size)+'/'
    results_df1.to_csv('/mnt/scratchc/fmlab/eudari01/BREAST_METABRIC/CIN_results' + nametosave_epoch)
    
    results_df2 = pd.DataFrame()
    
    nametosave = 'CIN_'+labeltouse  + '_lr_'+str(lr)+ '_momentum_'+str(momentum)+'_dropout_'+str(dropout)+'_batch_size_'+str(batch_size)+'/'
    results_df1.to_csv('/mnt/scratchc/fmlab/eudari01/BREAST_METABRIC/CIN_results' + nametosave)
    
    print('Results saved!!')
 
    
    
    


In [2]:
def main(job):
    runmymodel2(job)

if __name__ == "__main__":
    import sys
    main(sys.argv[1])

NameError: name 'pd' is not defined